In [1]:
!pip install opencv-python mediapipe gtts numpy



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np
from gtts import gTTS
import os

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

In [4]:
def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output.mp3")
    os.system("start output.mp3")  # Windows
    # os.system("mpg321 output.mp3")  # Linux/Mac


In [5]:
import cv2
import mediapipe as mp
import pyttsx3

# Initialize text-to-speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Adjust speed

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Initialize webcam
cap = cv2.VideoCapture(0)

def text_to_speech(text):
    """Continuously speaks the detected gesture."""
    engine.say(text)
    engine.runAndWait()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB for MediaPipe
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract key points
            thumb_tip = hand_landmarks.landmark[4].y
            index_tip = hand_landmarks.landmark[8].y
            middle_tip = hand_landmarks.landmark[12].y
            ring_tip = hand_landmarks.landmark[16].y
            pinky_tip = hand_landmarks.landmark[20].y
            wrist = hand_landmarks.landmark[0].y

            # Define gestures
            if (index_tip < wrist and middle_tip < wrist and 
                ring_tip < wrist and pinky_tip < wrist and thumb_tip < wrist):  
                gesture_text = "Hello"  # Open Palm Gesture
            elif (thumb_tip < wrist and index_tip > wrist and middle_tip > wrist and 
                  ring_tip > wrist and pinky_tip > wrist):  
                gesture_text = "Ok"  # Thumbs Up Gesture
            elif (index_tip < wrist and middle_tip < wrist and 
                  ring_tip > wrist and pinky_tip > wrist and thumb_tip > wrist):
                gesture_text = "Peace"  # Index and Middle Finger Up
            elif (index_tip > wrist and middle_tip > wrist and 
                  ring_tip > wrist and pinky_tip > wrist and thumb_tip > wrist):
                gesture_text = "Fist"  # All Fingers Down
            elif (index_tip < wrist and middle_tip < wrist and 
                  ring_tip < wrist and pinky_tip < wrist and thumb_tip > wrist):
                gesture_text = "Stop"  # All Fingers Up
            else:
                gesture_text = "Unknown"

            # Display Gesture Text
            cv2.putText(frame, gesture_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Speak the detected gesture continuously
            text_to_speech(gesture_text)

    cv2.imshow("Hand Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 